In [29]:
from elasticsearch import Elasticsearch
import json 
from math import cos, sin, atan2, sqrt, radians, degrees
import csv
from datetime import datetime, timedelta
from collections import defaultdict
import pandas as pd
import math


In [11]:
ELASTIC_USERNAME = "team11"
ELASTIC_PASSWORD = "v6Smlb2yINWc4" # INSERT YOUR PASSWORD

hosts = ["https://eestec2024.cl-test.medius.si:9200"]
# How many samples the query returns (maximum is 10000) if you want more use scroll
number_of_samples = 10000

# initialize client

In [12]:
# Connect to the Elasticsearch cluster
es = Elasticsearch(
    hosts,
    ssl_show_warn=False,  # Suppress SSL warnings, use with caution
    verify_certs=False,  # This skips SSL certificate verification
    basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD)  # For basic authentication
    # bearer_auth='Bearer <token>'  # For bearer token authentication, if applicable
)


In [22]:
indices = es.indices.get_alias(index="geo-locations-*")
for index in indices:
    print(index)

geo-locations-12.01.2023
geo-locations-10.01.2023
geo-locations-09.01.2023
geo-locations-11.01.2023


# Get data for specific gym

In [30]:
def calculate_square_coordinates(center_lat, center_lon, side_length):
    # Earth radius in meters
    earth_radius = 6371000
    
    # Convert latitude and longitude from degrees to radians
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)
    
    # Calculate the half side length of the square in meters
    half_side_length = side_length / 2
    
    # Calculate the change in latitude and longitude
    delta_lat = (half_side_length / earth_radius) * (180 / math.pi)
    delta_lon = (half_side_length / earth_radius) * (180 / math.pi) / math.cos(center_lat_rad)
    
    # Calculate the top left and bottom right coordinates
    top_left_x = center_lon - delta_lon
    top_left_y = center_lat + delta_lat
    bottom_right_x = center_lon + delta_lon
    bottom_right_y = center_lat - delta_lat
    
    return top_left_x, top_left_y, bottom_right_x, bottom_right_y





In [168]:
center_latitude = 46.05
center_longitude = 14.51
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 1

In [89]:
center_latitude = 46.061345
center_longitude = 14.541045
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 2

In [111]:
center_latitude = 46.059817
center_longitude = 14.518447
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 3

In [126]:
center_latitude = 46.245762
center_longitude = 15.278584
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 4

In [142]:
center_latitude = 46.076444 
center_longitude = 14.510828
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 5

In [182]:
center_latitude = 46.059904
center_longitude = 14.534631
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 6

In [201]:
center_latitude = 46.069197 
center_longitude = 14.543612
top_left_x, top_left_y, bottom_right_x, bottom_right_y = calculate_square_coordinates(center_latitude, center_longitude,500)
gym_number = 7

## DAY 1

In [202]:

# Define your search query
search_query = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "location": {
            "top_left": [top_left_x, top_left_y],
            "bottom_right": [bottom_rigth_x, bottom_rigth_y]
          }
        }
      }
    }
  },
  "size": 10000
}
# Execute the search query
response = es.search(index="geo-locations-12.01.2023",
                     body=search_query,
                     scroll='1m'  # Keep the scroll window open for 1 minute
                     )

# Initialize an empty list to store the search results
search_results = []

# Get the scroll ID
scroll_id = response['_scroll_id']

# Keep scrolling while we have data
while len(response['hits']['hits']):
    # Process hits here
    print("Received:", len(response['hits']['hits']), "hits")
    # Add hits to the search_results list
    search_results.extend(response['hits']['hits'])

    # Check if we have reached the maximum limit of 10,000 instances
    if len(search_results) >= 300000:
        break

    # Make a request using the Scroll API
    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # Extend the scroll window for another minute
    )

# Clear the scroll when you're done
es.clear_scroll(scroll_id=scroll_id)

# Define the filename to store the search results
day_date = "12.01.2023"
output_file = f"gym_{gym_number}_{day_date}.json"

# Write the search results to the output file
with open(output_file, "w") as f:
    # Write the search_results list to the JSON file
    json.dump(search_results, f)


Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits


In [203]:

# Define your search query
search_query = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "location": {
            "top_left": [top_left_x, top_left_y],
            "bottom_right": [bottom_rigth_x, bottom_rigth_y]
          }
        }
      }
    }
  },
  "size": 10000
}
# Execute the search query
response = es.search(index="geo-locations-10.01.2023",
                     body=search_query,
                     scroll='1m'  # Keep the scroll window open for 1 minute
                     )

# Initialize an empty list to store the search results
search_results = []

# Get the scroll ID
scroll_id = response['_scroll_id']

# Keep scrolling while we have data
while len(response['hits']['hits']):
    # Process hits here
    print("Received:", len(response['hits']['hits']), "hits")
    # Add hits to the search_results list
    search_results.extend(response['hits']['hits'])

    # Check if we have reached the maximum limit of 10,000 instances
    if len(search_results) >= 300000:
        break

    # Make a request using the Scroll API
    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # Extend the scroll window for another minute
    )

# Clear the scroll when you're done
es.clear_scroll(scroll_id=scroll_id)

# Define the filename to store the search results
day_date = "10.01.2023"
output_file = f"gym_{gym_number}_{day_date}.json"

# Write the search results to the output file
with open(output_file, "w") as f:
    # Write the search_results list to the JSON file
    json.dump(search_results, f)


Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits


In [204]:

# Define your search query
search_query = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "location": {
            "top_left": [top_left_x, top_left_y],
            "bottom_right": [bottom_rigth_x, bottom_rigth_y]
          }
        }
      }
    }
  },
  "size": 10000
}
# Execute the search query
response = es.search(index="geo-locations-09.01.2023",
                     body=search_query,
                     scroll='1m'  # Keep the scroll window open for 1 minute
                     )

# Initialize an empty list to store the search results
search_results = []

# Get the scroll ID
scroll_id = response['_scroll_id']

# Keep scrolling while we have data
while len(response['hits']['hits']):
    # Process hits here
    print("Received:", len(response['hits']['hits']), "hits")
    # Add hits to the search_results list
    search_results.extend(response['hits']['hits'])

    # Check if we have reached the maximum limit of 10,000 instances
    if len(search_results) >= 300000:
        break

    # Make a request using the Scroll API
    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # Extend the scroll window for another minute
    )

# Clear the scroll when you're done
es.clear_scroll(scroll_id=scroll_id)

# Define the filename to store the search results
day_date = "09.01.2023"
output_file = f"gym_{gym_number}_{day_date}.json"

# Write the search results to the output file
with open(output_file, "w") as f:
    # Write the search_results list to the JSON file
    json.dump(search_results, f)


Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits


In [205]:

# Define your search query
search_query = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "location": {
            "top_left": [top_left_x, top_left_y],
            "bottom_right": [bottom_rigth_x, bottom_rigth_y]
          }
        }
      }
    }
  },
  "size": 10000
}
# Execute the search query
response = es.search(index="geo-locations-11.01.2023",
                     body=search_query,
                     scroll='1m'  # Keep the scroll window open for 1 minute
                     )

# Initialize an empty list to store the search results
search_results = []

# Get the scroll ID
scroll_id = response['_scroll_id']

# Keep scrolling while we have data
while len(response['hits']['hits']):
    # Process hits here
    print("Received:", len(response['hits']['hits']), "hits")
    # Add hits to the search_results list
    search_results.extend(response['hits']['hits'])

    # Check if we have reached the maximum limit of 10,000 instances
    if len(search_results) >= 300000:
        break

    # Make a request using the Scroll API
    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # Extend the scroll window for another minute
    )

# Clear the scroll when you're done
es.clear_scroll(scroll_id=scroll_id)

# Define the filename to store the search results
day_date = "11.01.2023"
output_file = f"gym_{gym_number}_{day_date}.json"

# Write the search results to the output file
with open(output_file, "w") as f:
    # Write the search_results list to the JSON file
    json.dump(search_results, f)


Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits
Received: 10000 hits


In [206]:
for day in indices:
    # Load JSON data
    day_date = day.split("-")[-1] 
    input_file = f"gym_{gym_number}_{day_date}.json"
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)
    
    # Extract relevant information and track lowest and highest times for each MSISDN
    msisdn_data = defaultdict(list)
    for entry in data:
        msisdn = entry['_source']['msisdn']
        dateTimeEvent = entry['_source']['dateTimeEvent']
        msisdn_data[msisdn].append(dateTimeEvent)
    
    # Prepare filtered data
    filtered_data = []
    for msisdn, entries in msisdn_data.items():
        entries.sort()  # Sort entries by dateTimeEvent
        if len(entries) >= 2:
            time_arrived = datetime.strptime(entries[0], "%d.%m.%Y %H:%M:%S")
            time_left = datetime.strptime(entries[-1], "%d.%m.%Y %H:%M:%S")
            time_difference = (time_left - time_arrived).total_seconds() / 60  # in minutes
            if time_difference > 30 and time_difference / 60 <= 4:  # Remove instances less than 30 minutes and more than 4 hours
                filtered_data.append([entries[0], entries[-1]])
    
    # Write to CSV file
    output_file = f"gym_{gym_number}_{day_date}.csv"
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Time_arrived', 'Time_left'])
        writer.writerows(filtered_data)

In [207]:
# Function to round time to nearest 15-minute interval
def round_time_to_15_minutes(dt):
    return dt - timedelta(minutes=dt.minute % 15,
                          seconds=dt.second,
                          microseconds=dt.microsecond)

for day in indices:
    # Initialize dictionary to store counts for each 15-minute interval
    interval_counts = {}
    day_date = day.split("-")[-1] 
    
    # Define the date from the 'day' variable
    start_time = datetime.strptime(f"{day_date} 00:00:00", "%d.%m.%Y %H:%M:%S")
    end_time = datetime.strptime(f"{day_date} 23:59:59", "%d.%m.%Y %H:%M:%S")
    
    # Define 15-minute intervals throughout the day
    current_time = start_time
    interval_id = 1
    while current_time <= end_time:
        interval_counts[str(interval_id)] = 0
        interval_id += 1
        current_time += timedelta(minutes=15)
    
    # Parse the CSV file and count instances touching each interval
    input_file = f"gym_{gym_number}_{day_date}.csv"
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            time_arrived = datetime.strptime(row[0], "%d.%m.%Y %H:%M:%S")
            time_left = datetime.strptime(row[1], "%d.%m.%Y %H:%M:%S")
    
            # Round arrival and departure times to nearest 15-minute interval
            rounded_arrival_time = round_time_to_15_minutes(time_arrived)
            rounded_departure_time = round_time_to_15_minutes(time_left)
    
            # Increment counts for each interval touched by the instance
            current_time = start_time
            interval_id = 1
            while current_time <= end_time:
                if rounded_arrival_time <= current_time < rounded_departure_time:
                    interval_counts[str(interval_id)] += 1
                interval_id += 1
                current_time += timedelta(minutes=15)
    
    # Write the counts to a new CSV file
    output_file = f"gym_{gym_number}_{day_date}_processed.csv"
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['val', 'target'])
        for interval_id, count in interval_counts.items():
            writer.writerow([interval_id, count])


In [208]:
# Initialize an empty DataFrame to store combined data
combined_df = pd.DataFrame()

for day in indices:
    day_date = day.split("-")[-1] 
    # Read each processed CSV file
    df = pd.read_csv(f"gym_{gym_number}_{day_date}_processed.csv")
    
    # Concatenate the current DataFrame with the combined DataFrame
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv(f"gym_{gym_number}.csv", index=False)


# TRAIN MODELS

In [209]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV

data = pd.read_csv(f"gym_{gym_number}.csv")

X = data[['val']]
y = data['target']

## Grid seach for best paramethers for Random forest

In [210]:

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 500, 1000],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Create the grid search model
grid_search = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

# Get the best model
best_random_forest_model = grid_search.best_estimator_

# Evaluate the best model's performance
y_pred_best = best_random_forest_model.predict(X_test)

# Print evaluation metrics for the best model
print("Mean Squared Error (Best Model):", mean_squared_error(y_test, y_pred_best))
print("R-squared Score (Best Model):", r2_score(y_test, y_pred_best))


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 500}
Mean Squared Error (Best Model): 24979.93691554219
R-squared Score (Best Model): 0.6183685101634196


In [192]:
new_data = pd.DataFrame({'val': [20]})
best_random_forest_model.predict(new_data)

array([21.32963512])

## random forest model

In [193]:
random_forest_model = RandomForestRegressor(n_estimators=100,max_depth = 10, max_features = "sqrt", min_samples_leaf = 4, min_samples_split = 10,  random_state=42)  # You can adjust parameters as needed

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
random_forest_model.fit(X_train, y_train)

# Evaluate the model's performance
y_pred = random_forest_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared Score:", r2_score(y_test, y_pred))


Mean Squared Error: 24761.63342074806
R-squared Score: 0.6217036462062564


In [194]:
new_data = pd.DataFrame({'val': [20]})
random_forest_model.predict(new_data)

array([7.09648479])

## Linear model

In [195]:
from sklearn.linear_model import LinearRegression
#  Create a Linear Regression model
linear_regression_model = LinearRegression()
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model
linear_regression_model.fit(X_train, y_train)
# Evaluate the model's performance
y_pred = linear_regression_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared Score:", r2_score(y_test, y_pred))

Mean Squared Error: 58801.123228956174
R-squared Score: 0.10166465440636074


In [196]:
new_data = pd.DataFrame({'val': [20]})
linear_regression_model.predict(new_data)

array([173.27177072])

In [211]:
gym_number = 7
data = pd.read_csv(f"gym_{gym_number}.csv")
X = data[['val']]
y = data['target']

## Ada boost model

In [213]:
from sklearn.ensemble import AdaBoostRegressor

# Step 1: Create an AdaBoost model
ada_boost_model = AdaBoostRegressor(n_estimators=100, random_state=42)  # You can adjust parameters as needed

# Step 2: Train the model
ada_boost_model.fit(X_train, y_train)

# Step 3: Evaluate the model's performance
y_pred_ada = ada_boost_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error (AdaBoost):", mean_squared_error(y_test, y_pred_ada))
print("R-squared Score (AdaBoost):", r2_score(y_test, y_pred_ada))

# Step 4: Make predictions
# You can make predictions for new data using: ada_boost_model.predict(new_data)


Mean Squared Error (AdaBoost): 18682281.412359204
R-squared Score (AdaBoost): 0.0731565033709679


## Decision tree 

In [214]:
from sklearn.tree import DecisionTreeRegressor

# Create a Decision Tree model
decision_tree_model = DecisionTreeRegressor(random_state=42)  # You can adjust parameters as needed

# Train the model
decision_tree_model.fit(X_train, y_train)

# Evaluate the model's performance
y_pred_dt = decision_tree_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error (Decision Tree):", mean_squared_error(y_test, y_pred_dt))
print("R-squared Score (Decision Tree):", r2_score(y_test, y_pred_dt))

Mean Squared Error (Decision Tree): 26627897.32756133
R-squared Score (Decision Tree): -0.32103210107031677


#  Saving the trained model

In [212]:
from joblib import dump, load


random_forest_model = RandomForestRegressor(n_estimators=100,max_depth = 10, max_features = "sqrt", min_samples_leaf = 4, min_samples_split = 10,  random_state=42)  # You can adjust parameters as needed

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
random_forest_model.fit(X_train, y_train)

# Evaluate the model's performance
y_pred = random_forest_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared Score:", r2_score(y_test, y_pred))

#Save the trained model to a file
dump(random_forest_model, f'random_forest_model_{gym_number}.joblib')


#---------example case ----------------------
# Make predictions using the trained model
# Load the saved model from file
#random_forest_model = load('random_forest_model.joblib')

#new_data = pd.DataFrame({'val': [5, 10, 15]})
#predictions = random_forest_model.predict(new_data)

Mean Squared Error: 19054252.427384563
R-squared Score: 0.054702712391099695


['random_forest_model_7.joblib']

## Gradient Boosting regresson

In [215]:
from sklearn.ensemble import GradientBoostingRegressor

#Create a Gradient Boosting model
gb_reg = GradientBoostingRegressor(random_state=42)  # You can adjust parameters as needed

# Train the model
gb_reg.fit(X_train, y_train)

# Evaluate the model's performance
y_pred_gb = gb_reg.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error (Gradient Boosting):", mean_squared_error(y_test, y_pred_gb))
print("R-squared Score (Gradient Boosting):", r2_score(y_test, y_pred_gb))

# Make predictions
# You can make predictions for new data using: gb_reg.predict(new_data)


Mean Squared Error (Gradient Boosting): 22901437.08200324
R-squared Score (Gradient Boosting): -0.13615931343757826


## LASSO regression

In [216]:
from sklearn.linear_model import LassoLars

# Create a LassoLars model
lad_reg = LassoLars(random_state=42)  # You can adjust parameters as needed

#  Train the model
lad_reg.fit(X_train, y_train)

# Evaluate the model's performance
y_pred_lad = lad_reg.predict(X_test)

# Print evaluation metrics (if applicable for LassoLars)
# LassoLars does not directly provide an R-squared score, so you might want to use a different evaluation metric
print("Mean Squared Error (LassoLars):", mean_squared_error(y_test, y_pred_lad))


Mean Squared Error (LassoLars): 20179761.634689383
